Taxonomy
---
In this file we will assign taxonomy labels to the ASVs obtained from the denoising. In order to do that we will use the `classify-sklearn` action from the `feature-classifier` plugin and fetch:
- a pretrained classifier (from greengenes as the silva needs too much RAM)
- the sequences which we want to have classified (from the denoising step: dada2_rep_set.qza)
**Note:** As we don't know which primers were used we actually cannot just take a pre-trained classifier as such classifiers corresponds to a certain primer pair/rRNA region. But we do know the sequences (101bp) originate from the V4 region (300bp) of the 16S rRNA. As the most common primer pair (515f/806r) fully covers the V4 region we think it should be possible to use a classifier which was trained with 515f/806r seqeunces. Why not use a full lenght classifier? It was reported that Species-level classification performance of 16S rRNA gene simulated reads had a slightly lower accuracy in full-lenght sequences than in V1–3 and V4 subdomains. (from https://doi.org/10.1186/s40168-018-0470-z) 

In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline
data_dir = 'CE'

In [2]:
#fetch the pre-trained classifier 
! wget -nv -O $data_dir/515f-806r-classifier.qza https://data.qiime2.org/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza

2022-11-01 07:26:55 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza [28289645/28289645] -> "CE/515f-806r-classifier.qza" [1]


In [3]:
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_dir/515f-806r-classifier.qza \
    --i-reads $data_dir/dada2_rep_set.qza \
    --o-classification $data_dir/taxonomy_v4.qza

Saved FeatureData[Taxonomy] to: CE/taxonomy_v4.qza


This should have created a new taxonomy.qza artifact (containing our taxonomic assignments per feature)

In [4]:
! qiime tools peek $data_dir/taxonomy_v4.qza

UUID:        24a0655a-6f0c-4904-83fb-b1a2ee622b2d
Type:        FeatureData[Taxonomy]
Data format: TSVTaxonomyDirectoryFormat


Visualizations
---
We can make a tabular representation of all the features labeled with their corresponding taxonomy:

In [5]:
! qiime metadata tabulate \
    --m-input-file $data_dir/taxonomy_v4.qza \
    --o-visualization $data_dir/taxonomy_v4.qzv

Saved Visualization to: CE/taxonomy_v4.qzv


In [15]:
Visualization.load(f'{data_dir}/taxonomy_v4.qzv')

<visualization: Visualization uuid: 25326f1a-4573-4976-b1a6-a30b926b7b60>

By combining the taxonomic inforamtion with the metadata of our samples we can also get a hint of what the distribution of species is in the different samples! 
**Note:** dada2_table is a feature table from the denoising step which includes all ASVs (with error-corrected sequences)

In [8]:
#filter the feature table by excluding samples not present in metadata
! qiime feature-table filter-samples \
    --i-table $data_dir/dada2_table.qza \
    --m-metadata-file $data_dir/food-metadata.tsv \
    --o-filtered-table $data_dir/dada2_table_aligned.qza

Saved FeatureTable[Frequency] to: CE/dada2_table_aligned.qza


In [10]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_aligned.qza \
    --m-sample-metadata-file $data_dir/food-metadata.tsv \
    --o-visualization $data_dir/dada2_table_aligned.qzv

Saved Visualization to: CE/dada2_table_aligned.qzv


In [11]:
Visualization.load(f'{data_dir}/dada2_table_aligned.qzv')

<visualization: Visualization uuid: a58c588a-84b0-4329-8b2f-e99b37f1aef3>

In [12]:
! qiime taxa barplot \
    --i-table $data_dir/dada2_table_aligned.qza \
    --i-taxonomy $data_dir/taxonomy_v4.qza \
    --m-metadata-file $data_dir/food-metadata.tsv \
    --o-visualization $data_dir/taxa-bar-plots_v4.qzv

Saved Visualization to: CE/taxa-bar-plots_v4.qzv


In [13]:
Visualization.load(f'{data_dir}/taxa-bar-plots_v4.qzv')

<visualization: Visualization uuid: 905bd027-1cbf-4c2d-ba9c-8a255408bbb3>

To investigate how our taxonomic analysis compares to e.g. BLAST. We can open the tabular view of our ASVs and search for some features manually (click on sequence)

In [16]:
#filter feature table and exclude mitochondria,chloroplast
! qiime taxa filter-table \
--i-table $data_dir/dada2_table_aligned.qza \
--i-taxonomy $data_dir/taxonomy_v4.qza \
--p-exclude mitochondria,chloroplast \
--o-filtered-table $data_dir/dada2_table_align_filtered.qza

Saved FeatureTable[Frequency] to: CE/dada2_table_align_filtered.qza


In [17]:
#filter sequences and exclude mitochondria,chloroplast
! qiime taxa filter-seqs \
--i-sequences $data_dir/dada2_rep_set.qza \
--i-taxonomy $data_dir/taxonomy_v4.qza \
--p-exclude mitochondria,chloroplast \
--o-filtered-sequences $data_dir/dada2_rep_set_filtered.qza

Saved FeatureData[Sequence] to: CE/dada2_rep_set_filtered.qza


In [18]:
#this is the new barplot with the filtered deature atble and sequences NO MITOCHONDRIA AND CHLOOROPLAST VISIBLE
! qiime taxa barplot \
--i-table $data_dir/dada2_table_align_filtered.qza \
--i-taxonomy $data_dir/taxonomy_v4.qza \
--m-metadata-file $data_dir/food-metadata.tsv \
--o-visualization $data_dir/taxa-bar-plots_v4_filtered.qzv

Saved Visualization to: CE/taxa-bar-plots_v4_filtered.qzv


In [19]:
Visualization.load(f'{data_dir}/taxa-bar-plots_v4_filtered.qzv')

<visualization: Visualization uuid: d60898bf-ba86-4cb3-8734-27158f53f5ba>

**de novo tree**

In [ ]:
#Sequence alignment
 ! qiime alignment mafft \
     --i-sequences $data_dir/rep-seqs-filtered.qza \
     --o-alignment $data_dir/aligned-rep-seqs.qza

In [ ]:
#Alignment masking
! qiime alignment mask \
    --i-alignment $data_dir/aligned-rep-seqs.qza \
    --o-masked-alignment $data_dir/masked-aligned-rep-seqs.qza

In [ ]:
#Tree construction
! qiime phylogeny fasttree \
    --i-alignment $data_dir/masked-aligned-rep-seqs.qza \
    --o-tree $data_dir/fasttree-tree.qza

! qiime phylogeny midpoint-root \
    --i-tree $data_dir/fasttree-tree.qza \
    --o-rooted-tree $data_dir/fasttree-tree-rooted.qza

In [ ]:
#Tree visualization
 ! qiime empress tree-plot \
     --i-tree $data_dir/fasttree-tree-rooted.qza \
     --m-feature-metadata-file $data_dir/taxonomy.qza \
     --o-visualization $data_dir/fasttree-tree-rooted.qzv

In [ ]:
Visualization.load(f'{data_dir}/fasttree-tree-rooted.qzv')

**fragment insertion**

In [ ]:
! wget -nv -O $data_dir/sepp-refs-gg-13-8.qza https://data.qiime2.org/2021.4/common/sepp-refs-gg-13-8.qza

In [ ]:
! qiime fragment-insertion sepp \
    --i-representative-sequences $data_dir/rep-seqs-filtered.qza \
    --i-reference-database $data_dir/sepp-refs-gg-13-8.qza \
    --p-threads 2 \
    --o-tree $data_dir/sepp-tree.qza \
    --o-placements $data_dir/sepp-tree-placements.qza

In [ ]:
  ! qiime empress tree-plot \
      --i-tree $data_dir/sepp-tree.qza \
      --m-feature-metadata-file $data_dir/taxonomy.qza \
      --o-visualization $data_dir/sepp-tree-placements-tree.qzv

In [ ]:
Visualization.load(f'{data_dir}/sepp-tree-placements-tree.qzv')